# Sem Normalizacao

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib

In [2]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",
    "nWifi": 5,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:14774
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.057s)


Ns3Env parameters:
--nWifi: 5
--simulationTime: 60
--openGymPort: 14774
--envStepTime: 0.01
--seed: -1
--agentType: discrete
--scenario: basic
--dryRun: 0
Simulation started


Observation space shape: (1, 1)
Action space shape: (1, 1)


Simulation process id: 454267 (parent (waf shell) id: 454252)
Waiting for Python process to connect on port: tcp://localhost:14774
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
logger = teacher.eval(agent,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        tags=tags,
                        parameters=hyperparams)
# agent.save()


# %%

2022-12-01 18:06:41.783061: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


2022-12-01 18:06:41.965328: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3491555000 Hz
2022-12-01 18:06:41.967030: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55940ab852f0 executing computations on platform Host. Devices:
2022-12-01 18:06:41.967492: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





2022-12-01 18:06:44.792624: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.


...

Sent 2338.10 Mb/s.	Mean speed: 38.97 Mb/s	Episode 15/15 finished

Training finished.


COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-size/c8a1e4e4baca4bb89d408b0dd3aedba3

100%|██████████| 6300/6300 [15:24<00:00,  6.81it/s, curr_speed=39.17 Mbps, mb_sent=2338.10 Mb]


Waiting for simulation script to connect on port: tcp://localhost:46637
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.124s)


Ns3Env parameters:
--nWifi: 5
--simulationTime: 60
--openGymPort: 46637
--envStepTime: 0.01
--seed: -1
--agentType: discrete
--scenario: basic
--dryRun: 0
Simulation started
Simulation process id: 647673 (parent (waf shell) id: 647645)
Waiting for Python process to connect on port: tcp://localhost:46637
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-size/c8a1e4e4baca4bb89d408b0dd3aedba3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [5999]           : (32.0, 32.0)
COMET INFO:     Cumulative reward          : (3272.688720703125, 3272.688720703125)
COMET INFO:     Current throughput [5999]  : (38.36700439453125, 40.209999084472656)
COMET INFO:     Fairness index [5999]      : (0.9699320197105408, 0.9999369978904724)
COMET INFO:     Megabytes sent [5999]      : (0.40209999680519104, 2338.09754678607)

Sent 2338.10 Mb/s.	Mean speed: 38.97 Mb/s	Eval finished



# Execution num2: Com Normalizacao

In [ ]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

In [ ]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",
    "nWifi": 5,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

In [ ]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
logger = teacher.eval(agent,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        tags=tags,
                        parameters=hyperparams)
# agent.save()


# %%

# Execution num3: Com Normalizacao - Queue-GetNPackets: 100

In [ ]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

In [ ]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",
    "nWifi": 5,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

In [ ]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, state_size=5, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
logger = teacher.eval(agent,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        tags=tags,
                        parameters=hyperparams)
# agent.save()


# %%

# Execution num4:

In [ ]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

In [ ]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",
    "nWifi": 5,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

In [ ]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
logger = teacher.eval(agent,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        tags=tags,
                        parameters=hyperparams)
# agent.save()


# %%